In [4]:
import re
import string
import pandas as pd
import nltk
from nltk.collocations import *
from pattern.en import lemma
import requests
import json
import time
import hashlib
import uuid
import csv
import phrasemachine
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup
from urllib.request import Request
import difflib
from nltk.stem import WordNetLemmatizer 

# 1.Clean the text, lemma words

In [5]:
with open('The Heart Is a Lonely Hunter - Carson McCullers.txt', 'r',encoding='utf-8') as file:
    text = file.read().rstrip().lower()

In [6]:
# remove puntuation
word_list = re.findall("[a-z]+\-?\'?[a-z]+", text)

In [92]:
#Lemmatization
lemma_full_word_list = []
for i in word_list:
    word = lemma(i)
    lemma_full_word_list.append(word)
lemma_full_word_list

TypeError: 'LemmaDB' object is not callable

In [14]:
CET4_list = []

with open('CET4_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        if line.startswith(('a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z')):
            word = re.findall('([a-z]+)',line)
            CET4_list.append(word[0])
        else:
            continue

In [15]:
highschool_list = []

with open('Highschool_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        highschool_list.append(line.lower().rstrip())

In [16]:
elementary_list = []

with open('Elementary_edited.txt','r',encoding='utf-8') as fhandle:
    for line in fhandle:
        highschool_list.append(line.lower().rstrip())

In [17]:
easy_word_list = set(CET4_list) | set(highschool_list) | set(elementary_list)
easy_word_list = list(easy_word_list)


In [18]:
#Lemmatize easy word list
lemma_easy_word_list = []
for i in easy_word_list:
    word = lemma(i)
    lemma_easy_word_list.append(word)

In [19]:
#select unfamiliar words
raw_word_list = []
for word in lemma_full_word_list:
    if word not in lemma_easy_word_list:
        raw_word_list.append(word)
    else:
        continue    

In [20]:
#clean '
raw_word_clean_list = []
for word in raw_word_list:
    word_clean = re.findall("[a-z]+\-?[a-z]+", word)[0]
    raw_word_clean_list.append(word_clean)

    

In [21]:
#count the word and write a csv
word_count = pd.value_counts(raw_word_clean_list)
data = pd.DataFrame({'Vocabulary':word_count.index, 'counts':word_count.values})
data.to_csv("vocabularycounts.csv",index=False,header=True)

In [22]:
data

,Vocabulary,counts
0,jake,230
1,portia,214
2,mick,214
3,copeland,205
4,biff,182
...,...,...
2953,jeer,1
2954,peg,1
2955,lattice,1
2956,fingertip,1


# 2. Extact en translation and phonetics

#### 2.1 test

In [127]:
app_id = '761150e6'
app_key = '9c547ac3ff97fad8a7e366f8129d6a2c'

word_id = 'better'
strictMatch = 'false'

url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/en-gb/' + word_id.lower() + '?strictMatch=' + strictMatch;

r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
pron = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']
cate = r.json()["results"][0]['lexicalEntries'][0]['lexicalCategory']['text']
defin = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions'][0]
print(pron,cate,defin)

ˈbɛtə Adjective more desirable, satisfactory, or effective


In [79]:
a = r.json()
b = a["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']


'eɪs'

In [122]:
app_id = '5c864748'
app_key = '11013df45a50b3d7960bdcb4fe77af01'

language = 'en'
word_id = 'better'
strictMatch = 'false'

url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/' + language + '/' + word_id.lower() + '?fields=' + fields + '&strictMatch=' + strictMatch;

r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
pron = r.json()
pron

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [51]:
c = r.json()
d = c["results"][0]['lexicalEntries'][0]['lexicalCategory']['text']
d

'Adjective'

In [90]:
# Youdao translation for plan B
youdao_url = 'https://openapi.youdao.com/api'  

input_text = "better"
translate_text = "dream"
time_curtime = int(time.time())   # 秒级时间戳获取
app_id = "15a1da979e8d68c0"   # 应用id
uu_id = uuid.uuid4()   # 随机生成的uuid数，为了每次都生成一个不重复的数。
app_key = "SSfsA7MhSvqvG79kKVzGx875O2ukapUW"   # 应用密钥

sign = hashlib.sha256((app_id + input_text + str(uu_id) + str(time_curtime) + app_key).encode('utf-8')).hexdigest()   # sign生成


data = {
    'q':translate_text,   # 翻译文本
    'from':"en",   # 源语言
    'to':"zh-CHS",   # 翻译语言
    'appKey':app_id,   # 应用id
    'salt':uu_id,   # 随机生产的uuid码
    'sign':sign,   # 签名
    'signType':"v3",   # 签名类型，固定值
    'curtime':time_curtime,   # 秒级时间戳
}



r = requests.get(youdao_url, params = data).json()
r

{'RequestId': 'ebd0daa4-53bb-4eaa-a59b-93cdc3729ad9',
 'errorCode': '202',
 'l': 'en2zh-CHS'}

#### 2.2 the full version

In [3]:
csvFile = open("The Heart is a Lonely Hunter_VocabCount_v4.csv", "r")
reader = csv.reader(csvFile)

In [4]:
# API id and key
app_id = '761150e6'
app_key = '9c547ac3ff97fad8a7e366f8129d6a2c'

In [5]:
word_pron_cate_defin = []    

In [6]:
for i in reader:
    try:
    
        word_id = i[0]
        strictMatch = 'false'

        url = 'https://od-api.oxforddictionaries.com:443/api/v2/entries/en-gb/' + word_id.lower() + '?strictMatch=' + strictMatch;

        r = requests.get(url, headers = {'app_id': app_id, 'app_key': app_key})
        pron = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['pronunciations'][0]['phoneticSpelling']
        cate = r.json()["results"][0]['lexicalEntries'][0]['lexicalCategory']['text']
        defin = r.json()["results"][0]['lexicalEntries'][0]['entries'][0]['senses'][0]['definitions'][0]
        word_pron_cate_defin.append([i[0],'/'+pron+'/',cate,defin])
        time.sleep(30)
    except KeyError:
        pass
    continue

In [27]:
for i in word_pron_cate_defin:
    if i[2] == 'Noun':
        i[2] = 'n.'
    if i[2] == 'Adjective':
        i[2] = 'adj.'
    if i[2] == 'Verb':
        i[2] = 'v.'
    if i[2] == 'Adverb':
        i[2] = 'adv.'


In [28]:
fileObject = open('The Heart is a Lonely Hunter_VocabCount_v5.txt', 'w',encoding='utf-8')
for i in word_pron_cate_defin:
    for j in i:
        fileObject.write(j)
        fileObject.write('|')
    fileObject.write('\n')
fileObject.close()

# 3. Find phrases in text

#### 3.1 test

In [104]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = BigramCollocationFinder.from_words(word_list)
scored = finder.score_ngrams(bigram_measures.raw_freq)
sorted(bigram for bigram, score in scored)

[('-', '-lamming'),
 ('-', '-legs'),
 ('------', 'she'),
 ('-lamming', 'his'),
 ('-legs', 'and'),
 ('-t', 'or'),
 ('a', 'and'),
 ('a', 'animal'),
 ('a', 'b'),
 ('a', 'babble'),
 ('a', 'baby'),
 ('a', 'backfire'),
 ('a', 'backwash'),
 ('a', 'bad'),
 ('a', 'baggy'),
 ('a', 'ball'),
 ('a', 'band'),
 ('a', 'bandage'),
 ('a', 'banjo'),
 ('a', 'bar'),
 ('a', 'barbecue'),
 ('a', 'bare'),
 ('a', 'barefooted'),
 ('a', 'barking'),
 ('a', 'basket'),
 ('a', 'batch'),
 ('a', 'bath'),
 ('a', 'bathrobe'),
 ('a', 'battle'),
 ('a', 'bay'),
 ('a', 'beaming'),
 ('a', 'beard'),
 ('a', 'beautiful'),
 ('a', 'beauty'),
 ('a', 'bed'),
 ('a', 'bedbug'),
 ('a', 'bedlam'),
 ('a', 'bedroom'),
 ('a', 'bell'),
 ('a', 'bellboy'),
 ('a', 'bellyful'),
 ('a', 'bench'),
 ('a', 'berry'),
 ('a', 'better'),
 ('a', 'bible'),
 ('a', 'bicycle'),
 ('a', 'big'),
 ('a', 'bilious'),
 ('a', 'bill'),
 ('a', 'bird'),
 ('a', 'bit'),
 ('a', 'bitter'),
 ('a', 'black'),
 ('a', 'blacksmith'),
 ('a', 'bland'),
 ('a', 'blankness'),
 ('a', 

In [159]:
text = "The Omnibus Crime Control and Safe Streets Act of 1968 was signed into law by President Lyndon B. Johnson.so as James"
phrasemachine.get_phrases(text)

Exception: We don't support tagger nltk

# 4. Scrape Booklist

#### bookerprize

In [134]:
url = []
for i in range(2010,2020):
    url.append('https://thebookerprizes.com/fiction/backlist/'+str(i))

In [147]:
name_author_list = []
for i in url:
    hdr = {'User-Agent': 'Mozilla/5.0'}
    req = Request(i,headers=hdr)
    html= urlopen(req)
    soup = BeautifulSoup(html, 'html.parser')
    shortlist = soup.find('div',class_="flexslider bookCarousel")
    name_author=shortlist.find_all('h4',class_="title")
    for k in name_author:
        try:
            name = k.get_text()
            author = re.findall('\-by\-(\S+)',k.a['href'])[0].replace('-',' ')
            name_author_list.append([name,author])
        except IndexError:
            pass
        continue

18


In [148]:
name_author_list

[['Parrot and Olivier in America', 'peter carey'],
 ['Room', 'emma donoghue'],
 ['In a Strange Room', 'damon galgut'],
 ['The Long Song', 'andrea levy'],
 ['C', 'tom mccarthy'],
 ['Pigeon English', 'stephen kelman'],
 ['Half Blood Blues', 'esi edugyan'],
 ['Jamrach’s Menagerie ', 'carol birch'],
 ['The Sisters Brothers', 'patrick dewitt'],
 ['The Lowland', 'jhumpa lahiri'],
 ['The Testament of Mary', 'colm toibin'],
 ['We Need New Names', 'noviolet bulawayo'],
 ['The Sellout', 'paul beatty'],
 ['Hot Milk ', 'deborah levy'],
 ['His Bloody Project ', 'graeme macrae burnet'],
 ['Eileen', 'ottessa moshfegh'],
 ['All That Man Is', 'david szalay'],
 [' Do Not Say We Have Nothing', 'madeleine thien']]

In [167]:
word = 'https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcaJnXUm70lia7Ot4h9L21OZFUzeNic40wicevPiajFuXlf3K65PNgiaPbtzg/0?wx_fmt=jpeg'
defination = 'https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcxUoRJ1YErqXYmesqTaxPnpJ7icDbyLat6tDUlQIeHFpo0fEdOcXJtFw/0?wx_fmt=jpeg'
css = """<section data-width="100%" style="display: inline-block; width: 100%; font-size: -3px; background-image: url(&quot;""" + defination + """&quot;); background-size: cover; vertical-align: top; background-attachment: scroll; box-sizing: border-box; text-align: center; background-position: 0% 0%; background-repeat: no-repeat; max-width: 100% !important;">
<svg data-width="100%" enable-background="new 0 0 1067 212" space="preserve" style="display: inline-block;outline: none;width: 100%;margin-top: -1px;border-width: initial;border-style: none;border-color: initial;vertical-align: top;background-position: 0% 0%;background-repeat: no-repeat;background-size: 100%;background-attachment: scroll;background-image: url(&quot;""" + word + """&quot;);-webkit-tap-highlight-color: transparent;" viewBox="0 0 1067 212" x="0px" xml="" y="0px">
<set attributeName="visibility" from="visible" to="hidden" begin="click+0.5"></set>
<animate attributeName="opacity" begin="click" dur="0.5s" values="1;0" restart="never" fill="freeze">
</animate>
<set attributeName="visibility" from="hidden" to="visible" begin="click+3"></set>
<animate attributeName="opacity" begin="click+3" dur="0.3s" values="0;1" restart="never" fill="freeze">
</animate>
</svg></section>"""
print(css)

<section data-width="100%" style="display: inline-block; width: 100%; font-size: -3px; background-image: url(&quot;https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcxUoRJ1YErqXYmesqTaxPnpJ7icDbyLat6tDUlQIeHFpo0fEdOcXJtFw/0?wx_fmt=jpeg&quot;); background-size: cover; vertical-align: top; background-attachment: scroll; box-sizing: border-box; text-align: center; background-position: 0% 0%; background-repeat: no-repeat; max-width: 100% !important;">
<svg data-width="100%" enable-background="new 0 0 1067 212" space="preserve" style="display: inline-block;outline: none;width: 100%;margin-top: -1px;border-width: initial;border-style: none;border-color: initial;vertical-align: top;background-position: 0% 0%;background-repeat: no-repeat;background-size: 100%;background-attachment: scroll;background-image: url(&quot;https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcaJnXUm70lia7Ot4h9L21OZFUzeNic40wicevPiajFuXlf3K65PNgiaPbtzg/0?wx_fmt=jpeg&quot;);-webkit-tap-highlight-

# 5. CSS editor

In [169]:
url = 'https://mp.weixin.qq.com/cgi-bin/filepage?type=2&count=12&begin=60&group_id=1&view=1&token=1151994297&lang=zh_CN'
html= urlopen(url)
soup = BeautifulSoup(html, 'html.parser')

In [194]:
pic = soup.find_all('div',class_ = "weui-desktop-media__list-wrp")

In [201]:
soup

<!DOCTYPE html>
<html><head> <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><meta content="webkit" name="renderer"/><meta content="width=device-width,initial-scale=1,user-scalable=0" name="viewport"/><meta charset="utf-8"/><script type="text/javascript">

window._points = [+new Date()];


function wx_main(mod) {
  window._points && (window._points[3] = +new Date());
};

window.wx = {
  uin: "" || "0",
  lang: "zh_CN"
};
</script><script crossorigin="" onerror="wx_loaderror(this)" src="https://res.wx.qq.com/mpres/zh_CN/htmledition/pages/modules/common/badjs478dab.js" type="text/javascript"></script><script crossorigin="" onerror="wx_loaderror(this)" src="https://res.wx.qq.com/mpres/zh_CN/htmledition/pages/modules/base/jserr445214.js" type="text/javascript"></script><script type="text/javascript">


window.wx_loaderror = function(ele) {
  var _img = new Image();
  var key = 63;
  if (ele && ele.tagName && ele.tagName.toLowerCase() == "script") {
    key = 64;
    
  }
  _

In [226]:
json = soup.find_all('script')[10]

In [233]:
a = json.get_text()
fileObject = open('wechat.txt', 'w',encoding='utf-8')
fileObject.write(a)
fileObject.close()

In [256]:
word = ['https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc3QUBMXNvQQsQoRjxU6xmYqV7OLH7CylgWsE73cea7YfAOVdJ1ib2JPA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcoRsMRo9U9bXFoM5yhyyvcfcJatZQINibBqTEBobkeqp4v5Zbzciav8Eg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzciczHtTAyiabKrJOpOpBsgQXW8jZ8yCh6BzedrOqfqBcFQnoYWWZh5iagw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc9EXBvk84NJT3iaKNhAHW111MIbfdaA5ytghGHfbYqFosGVNkGereyDQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcUHSkglkhq77XkAYm02RXEN0mH01G4tNBiaxbvHVzCALWf7NgEgVxRbA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcVTFobGcatIqTTicWmSwx7NpPufC5wQKeFwcQCKJL3FspprohZD4euxw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcR8FbRLmJQwhkqySodvbiaNoicc5oib54dLY5HnTIAxHfHd6EnOMISgThw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzczBNnA8m1DCElHibHJKhTWPP6yEkgT4WngjhJZiapwFfOmzNUZkrpicppQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzca8ueIPtubKPtQxdXcJDHn1jiaOLNBHT3tXwJJDXeARKbMW1xSMwT6SQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcKSicwUUq4QHricCn2PPKvhwx2DRxfEE4dUxxFLcdEzK1fzu1OFwibNmhg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzch3O59Z09kPr7wG4ibo3G3IFQmlJlcA6bibS1JIemPsWLCdgpPksmUJVA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcULSKL9I2ecEAuXx6agXZHgN8Ymxc5QGF7Ez31aRJImNhQYGCajvfEw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcqCpTFSOJqe29c2V8bG0nug38ic16XDsLgCsgAPzWrC6raaTtmjEyO6Q/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc4P8RaKeZvAX8XDF7rhAUTCX4sH9kaVyVpteN7jLQwRuvDX6Jt7jpLw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc4jFfucS2ASnia2GnqYMhycqibWeElt0yINo0HU4XsWxM1ASwD80L3UzA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc81zeDhfpUVLD2icF8NxXh7wBRlPMEDiananOQiatyUzCpSnyvGWPdicScQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcBEgR4iaNh8RbuMWrRL3E8t8yX4Jk4etfg74PQm6OVwowSRhYR07UTjw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcuz8le31CVjmHTk4xicCydDmkic6W3Ey0IGtbHS6N04riaKndMwRzOQINA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcuI6RyqmfOrPY9DAXH20Nsic9fzjITGQl821lOXz3WuClg1D5xbZmaNw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcY9I2fjHl2GTBvZhzaG9bI2uiccIGPKMicoKs7ia8Fs3b3fVA1zYFQ6Dyw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcqaCO8gKpeiaPpTJDFoqtPcq4S0TNOyjRkY1ZViahPq9ZsANr5wHHaPyw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzccqCmhhLe1082mbvHhtXPkdTfwiaNSgfApjwka1A0oTicyicFib0hO3L0eQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcEicSJv2ynIKYX5J6ib5fqCjuNicIVeuGMAvkylUBzeiamLXbAx1ZboJj5A/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcn8EUreScs5NPmZ7kkEwVV9zgpicoWibf4WNzplPFiapkloFNH5xWvwbeA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcs89lmBficmVHnvxibDTylMcOW6tyWuxnJctywOvwtHoDiafxyBSg8yFzQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc37XkHWWjbJWhy4pqu7VSlqB2fkut6pfnn1dFuNoDthmygBhBj1Xt7g/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcg00gw7domajRIggMmic27pqpcdquMouK3g1uPibIOibLgmEEBMgopKDRQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcdnLnut0vBN3wf3PJ7fDWyS3IRjXHRWauQuGroxG1awwRWOINb03Jrg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcO0mSVURtkGd9TINjWs4mnZeQA0iciaL6Ink9JbbvcPrFgbbPZ5PEPp5w/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcxkAibFDDDQrFtkDvUePUuicvBJcby7q7uWcIibgUCLbdyl5Gjla8WnRrw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzczWrx7rkHJnBomsEhquRwbJLwqiaxeoM1vg8QSFVMf68UIL0G3iaUu8IA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcrygaNeLkGa8Nt5CQ4vI0SOKawIB1Ap7IAOicpticITPw4ouZicwqiaHZxg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcLsYZeFMVwLg4Ixmibcf2ZQuWK5iccfcSGJ3efqUyXCr9IFRxxVYrgdAw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcs6V6kMw9rTUaA9cWiaJjcZJtjOwtnn84tXicelB1UMBc2xml9Gwmw37g/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzccZZdwnRtibTRnhB0nF1qWNU0oXIRQiawhPo7S3RMPkGCunADMZSNm2QQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcSic5kaCx7133cVY5aEuV6icJ0bqsKjnBHSe7icQ4dzKjNf2rFQnOhogibA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcLSWUxymYicY7KpvT4r4gLSBtiaqiaK7t4MFZtjmIUThLsdIwrjEM3CAgg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcA6kP85WrcBYiasARIp31sIBLkz6ianyxrUNjM4gMfph2XicHkOaUiaPazw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcY0HcDNhdtlOnWaLpVYticPhVzI0IfqkYlxbtmpk3icprqcmMIHSKbAqg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcdnrjs49VtwbLUo67sPxnMflsuQzHjPzU0CmcmluyE4NwMHcMp3pW2Q/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcFzOiay1IXma7HltPHqz2vC6icIfgDSSXcXGBfpibSu6q4Xnvqlr8rVHHA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcN0supfkBqhQvLiboVnv61YUaUcARt4ZRBqaYLTNiaqbfibAsKpGJwzgYw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcBsgT2iamsfJ3pLo32suyVl92gibBWTba5LnIibyYpzfOR9Kmvurodarnw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcempTGtN1dicxJJkibg0ibWrAuSBaCmAu98gscU3RxoYgK2qrO5XaXE7Tw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcbO0csKBJjsbEJEkLUjz9hKn6vicHtJSsvYUNfcgOemIJju9WQfgxibYg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcU3PheZ8MfusdxpEicicyfyWX0sScPt25cWsTUaXcMBoiaAzEEl9sCrGBw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcYe0HIlrP3wxicyyoZXeyVmkhaYicNibPbrTjQ9j0oJZnIfiaxYzbVU17KQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcAzQ8DpvS0EM9XUibEVQ5nPkeLveibDo7AMsVvYB1nWgQeJibbtfCh78pQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcalIxDPRasBYqhPwqWVZMktNHN0Wvhnamumknt3ic1tGEiacSI0QqjnHw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcrpBwZ9wU7TeSJOkP6rBxVUEYyCmeBnL0icBstMCrKnQhtGIqnnqfJoA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcTCO4kHwNqVDbHVR40ayEoqhL9ZvPBvkdic4V6wd5czhaFJ7I4FibvNuA/0?wx_fmt=jpeg']

In [257]:
definition = ['https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcS7yibsjaNibbzgR3GGg9Ty440hbOqqmb1CFRvwJWJOflriaONneqcFOFg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc8tCm8IjuJIBLj5RHJiarXTgAJRYBBUldBtbiaasaxcxIAMz8MsoiaFL9w/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcVdpTB2qR8w9zeLicO3e6QBGDRCsch1nKlRxxjk4UQRwKpB3nSlUhlAQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcfTkUBkjicibPNVrU0KvC9mlBqsoQHlPI61J6dnpqCbicyRUic3hiaeuhmTw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcYhbGmOHwvOlCf34aFib8lF9HQPJNkjYZlzjPn8TQbWI5IC5rMpRUA2A/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcl8YvrF0wI73A73Xs87Rgkh0kL0kgjicWIMOwPia3wXF1hR7nkddm0O8Q/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcZ1SYfUcEy5UwUAsrw9twjTS1PHWQSWsBZia2BiaavUDCiajsgbmu4uSIw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcrg7VRqicicQTVj9yQGCDR6AesCmnCib2668VW4y13WUKNOeoT2ccngUtw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzckzQaib3qGPngTMU0JtaBMJMvEicThhCqO9bSgXfhVtLZUdFvyCgOHoBQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc3gBESlFPdujgt6vbfibolS3u19HXLErDm9jBvmggqXfBLwAoRSrxoqQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcicw1n2GFLFmrGAbgOsPrE5hNXll3KSTXOu5ydgDw33qhYzpb4tz4RHw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc9XKiaicr4UVaLSI5libibKiazRqoHQeBbnZfx2Ive3ucJuP8NbwXa67kcQQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcxicWYpoOQGVrXKjyNdPKeTPu4MibJB7CgZ5Fkxhq4VITwUhzD1CGRIhQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcicEiaKN3Fib0vhoMpFZgbNBxm6IcA28BWXZbBhdAkyTEDoXaHcBWABVag/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcUhQZ1fdvfUHyndPlIe7jNNU38c36IibI2lLMP8Fv6dvc2oPfs1FGH2A/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcrFxcUPd3FcV3dUqviaPWA0kaQzlLBTp6PQLdZeUdFib3PeTJTCvVlRCw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzclDZiaRs5V0GLib7kQF4Ut0VNxGzCiaQhmpg4wF94aPPOFia5YANm6cMLMQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcQdcNLzm3JaL32bMQdnVr6MvR7dkGHBH0nAOqg4ibIQm3cia1bDxn8x8Q/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc09icNKFRVdYBdA9JDlVf3HicRAV8Xf0QcNNick9Zc3xScVQGRvA0ZGRcg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcRpibt8mAoLAoSovXWAQIsbnNpJ4rzBY5wHWXM6EiabbHw3H3rVRZ8rBA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcpPTNay2dXmkWQ3HT6eUwXoZDCG89HZX06uAtlrRw6tcybdKMIQSZqg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcFjLR1m6JR7OHia5c6Ec815xL0tmFFoibhoLQH2MtsTibRRyun1NyTXiaRQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc0xfo171aaCbcicSQUCthiaywicHDZWATl730PnxCeZIz6E5Z4I843k78g/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzchvg873GRM8XdwgDjgZxWgpKjSnLicpsaicic539iahnSwAyria4rPPy2FDw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcBDb1Jg4s7NBd7Kiau9Jvlic77ibz8pd98f9JuY8juQKY7UmIRmSRZ2cjg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc7KUe7tx9MZRJ6kCABaaYeJAlicibdqzzUakAaB5aK6PDG3Gk6TZITaFA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcehv7YYkraxWnhCU2v6RTQGPoKyPzia1IwtiaC4fbkVPxd87QX7HxEibMg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcQpMDZuXMcQN0NoAxq36qXgChPic4xGiabuD5hetyfX2K6bew1dAuZt7g/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcwUX8WDwKt5CHkVcIk0WIH1YKzR1PtLyoSW8EaHsoNlogM9jQ3fEyNw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcwCfRwICDtyibibPSRMuPiaK8YsO488uSLJmJ1XyJJxsz8rKTVKABbXTkw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcicy0CgF4fRrxg0KwG1ykXhJKo4S04bM1f3LT3OibBoqg0vhTckb0utcg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcwYBvefUGRwFpGhWheLrdeCg0IW7Q4foI6r6HMC9x98ibDV2UWc5sEAQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcoiaETF4DibB0EOiafia1sA9PjuJS5R7VGeQiawqzpPg2QTrxxF8NbVxFzmQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc1wh29K6BauSIVDm8U0XvtNibGewKiaGkialkr4oQcibhiaiciagC49YgILnnA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcJnwrhmGoAakiaFQPqR1C76mVMqHqqquZ0TD3GU9OjbvhwvjL0cvBjiaA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc1BAxxqUiawHlKQghnrkB4sNic09z6QXStWHHFtvS546o2t9AAhR0aHxg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcUKvxTXQcaNhdtHiavlFD1V9zAM52JQD4C6W5sat2ArKAsGgnlr2NeBQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcKOksAte1xAiaMH9vWj8Lt5LmkJsPfknA0oCpajfE2A4CtZzGDAqXkzw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzclicmEZvDicIQ5TOibM86lz1B6d9TlZEIEahleyibIdHNNaUl5qGlvag2cg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc8rRS0DCibibEwcEkg0VvVPYm711vbseuEAWqS8oYkYAqjGHd6FGy7yEQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc6BQxernJqpsCIYic5BibZRmHYvKRl6R1jwibHoFlPXb5rdTjBIcSWbHNA/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc1rgN5w5IwKIIy6b8917Nicfe0TjdJz1rYJCtN8LSlCHGuRllvrseupw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcaP6zMhBDcUorRvAHdmkgOXjVccWlMajwZJzPsDywhgxe1skMQMia8IQ/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcC3RmtSNUiajmGXMFx5TAtFxL4NFmOhw2HfyicFEBZCKib4RDyr6bj1Xxg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcdOz0G6o9lEd15ejsg37ZaqZXh0UWIyCuAEjAJ6jtXckFl3ssEdKGOg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcrqd8LW8f3ppESAtcibS4qqNRKJ3qlk8pyaZw3j9uDLsS29Ss8YIcADw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcaSR8AU55h7kbEEqtYERBmB7opU5SdnoUIjibCpB7ia4g4Q2mDDtnIfUg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcb6SVQ266MSzzszLNLliaNibCceicAosJ93wA4XW3BwOppZ6DU3FAib00yg/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzc8Ziag2ULLQKaoD7LrGFwMZEPVmJWTue4QH08M0FmZWCTmQiaaCUnKIDw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcf6Yeqe5UmXcf1QQytoZYgy7hr7W4tiaAKu1f59tBuBNrNw1op9huUpw/0?wx_fmt=jpeg','https://mmbiz.qpic.cn/mmbiz_jpg/B0OT7bl3qeH6X79DA4grDtmR9MYSCWzcRNugIeibO2hIzdBFIlpmlmYY6UtoxBurFkXsJf4BUhCANNYehw9TMsg/0?wx_fmt=jpeg']

In [258]:
len(word)

51

In [259]:
len(definition)

51

In [265]:
fileObject = open('wechat.txt', 'w',encoding='utf-8')
for i in range(0,51):
    css = """<section data-width="100%" style="display: inline-block; width: 100%; font-size: -3px; background-image: url(&quot;""" + definition[i] + """&quot;); background-size: cover; vertical-align: top; background-attachment: scroll; box-sizing: border-box; text-align: center; background-position: 0% 0%; background-repeat: no-repeat; max-width: 100% !important;">
<svg data-width="100%" enable-background="new 0 0 1067 212" space="preserve" style="display: inline-block;outline: none;width: 100%;margin-top: -1px;border-width: initial;border-style: none;border-color: initial;vertical-align: top;background-position: 0% 0%;background-repeat: no-repeat;background-size: 100%;background-attachment: scroll;background-image: url(&quot;""" + word[i] + """&quot;);-webkit-tap-highlight-color: transparent;" viewBox="0 0 1067 212" x="0px" xml="" y="0px">
<set attributeName="visibility" from="visible" to="hidden" begin="click+0.5"></set>
<animate attributeName="opacity" begin="click" dur="0.5s" values="1;0" restart="never" fill="freeze">
</animate>
<set attributeName="visibility" from="hidden" to="visible" begin="click+3"></set>
<animate attributeName="opacity" begin="click+3" dur="0.3s" values="0;1" restart="never" fill="freeze">
</animate>
</svg></section>"""
    fileObject.write(css)
fileObject.close()
    

# 6. Similar words

#### test

In [47]:
csvFile = open("The Heart is a Lonely Hunter_VocabCount_v4.csv", "r")
reader = csv.reader(csvFile)
match_word_list = []

In [48]:
for i in reader:
    match_word = difflib.get_close_matches(i[0], set(word_list),10,0.2)
    match_word_list.append(match_word)   

In [51]:
with open("The Heart is a Lonely Hunter_VocabCount_v7.csv","w",newline = '') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(match_word_list)

In [61]:
difflib.get_close_matches('prop', ['plup','prop','happy','pp','plop','pulp'],10,0.2)

['prop', 'plop', 'pp', 'pulp', 'plup', 'happy']

# 7. Phrase

#### test

In [27]:
phrase_list=[['bring','out'],['look','in','the','eye'],['dream','of'],['want','to'],['turn','out','of']]
raw_phrase_list = []

In [28]:
for i in range(0,len(word_list)):
    for j in phrase_list:
        if word_list[i] == j[0] and j[-1] in word_list[i+1:i+5]:
            if j[-2] in word_list[i:i+4]:
                raw_phrase_list.append(j)
            else:
                continue                
        else:
            continue
    

In [31]:
phrase_count = pd.value_counts(raw_phrase_list)
data = pd.DataFrame({'Phrase':phrase_count.index, 'counts':phrase_count.values})
data.to_csv("phrasecounts.csv",index=False,header=True)